Imports

In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr
from utils.helper import Helper
helper = Helper()

# Load the users and the recommendations from traditional aggregation methods
user_profiles = helper.load_data('results/user_profiles.pkl')
groups_exp_1 = helper.load_data('results/groups_exp_1.pkl')
groups_exp_2 = helper.load_data('results/groups_exp_2.pkl')
recommendations_exp_1 = helper.load_data('results/recommendations_exp_1.pkl')
recommendations_exp_2 = helper.load_data('results/recommendations_exp_2.pkl')


C:\Users\jiska\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jiska\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\jiska\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Functions

In [2]:
# Get the user profiles containing tag ratings of each group
def get_groups(groups_exp):
    groups = {}
    for g, group in enumerate(groups_exp):
        members = {}
        for user in group:
            for id in user.keys():
                members[id] = user_profiles[id]
        groups[g] = members
    return groups

# Create matrix of users and tag ratings for computing similarity between users
def create_matrix(groups):
    result = {}
    for group_id, group in groups.items():
        # Step 1: Collect all unique strings for the current group
        unique_strings = set()
        for ratings in group.values():
            unique_strings.update(ratings.keys())
        
        # Convert the set to a list to use as columns in DataFrame
        unique_strings = list(unique_strings)

        # Step 2: Create mapping of user ids to row indices
        users = list(group.keys())
        user_index = {user: idx for idx, user in enumerate(users)}

        # Step 3: Initialize an empty matrix
        num_users = len(users)
        num_strings = len(unique_strings)
        matrix = np.full((num_users, num_strings), np.nan)  # Using np.nan for missing ratings

        # Step 4: Populate the matrix
        string_index = {string: idx for idx, string in enumerate(unique_strings)}
        for member, ratings in group.items():
            for string, rating in ratings.items():
                row_idx = user_index[member]
                col_idx = string_index[string]
                matrix[row_idx, col_idx] = rating

        # Convert to pandas DataFrame for better readability
        df = pd.DataFrame(matrix, index=users, columns=unique_strings)
        
        # Store the DataFrame in the result dictionary
        result[group_id] = df
        
    return result

# Similarity functions
def cosine_similarity(matrix):
    num_users = matrix.shape[0]
    similarity_matrix = np.zeros((num_users, num_users))

    for i in range(num_users):
        for j in range(num_users):
            if i != j:
                similarity_matrix[i, j] = 1 - cosine(matrix[i], matrix[j])
            else:
                similarity_matrix[i, j] = 1.0  # Similarity with itself is 1

    return similarity_matrix

def pearson_similarity(matrix):
    num_users = matrix.shape[0]
    similarity_matrix = np.zeros((num_users, num_users))

    for i in range(num_users):
        for j in range(num_users):
            if i != j:
                valid_indices = ~np.isnan(matrix[i]) & ~np.isnan(matrix[j])
                if np.sum(valid_indices) > 0:
                    similarity_matrix[i, j], _ = pearsonr(matrix[i, valid_indices], matrix[j, valid_indices])
                else:
                    similarity_matrix[i, j] = 0  # If no valid ratings overlap, similarity is 0
            else:
                similarity_matrix[i, j] = 1.0  # Similarity with itself is 1

    return similarity_matrix

def compute_group_similarities(groups_matrix):
    similarity_results = {}

    for group_id, df in groups_matrix.items():
        matrix = df.to_numpy()

        # Compute cosine similarity
        cosine_sim = cosine_similarity(matrix)
        cosine_sim_df = pd.DataFrame(cosine_sim, index=df.index, columns=df.index)

        # Compute pearson similarity
        pearson_sim = pearson_similarity(matrix)
        pearson_sim_df = pd.DataFrame(pearson_sim, index=df.index, columns=df.index)

        similarity_results[group_id] = {
            'cosine_similarity': cosine_sim_df,
            'pearson_similarity': pearson_sim_df
        }

    return similarity_results

Analysis

In [3]:
groups = get_groups(groups_exp_1)
groups_matrix = create_matrix(groups)
group_similarities = compute_group_similarities(groups_matrix)

In [4]:
group_similarities

{0: {'cosine_similarity':         11176   52448   64625   37036   288146
  11176      1.0     1.0     1.0     1.0     1.0
  52448      1.0     1.0     1.0     1.0     1.0
  64625      1.0     1.0     1.0     1.0     1.0
  37036      1.0     1.0     1.0     1.0     1.0
  288146     1.0     1.0     1.0     1.0     1.0,
  'pearson_similarity':           11176     52448     64625     37036     288146
  11176   1.000000  0.058479  0.068283  0.085263 -0.059640
  52448   0.058479  1.000000 -0.035605  0.028293 -0.066133
  64625   0.068283 -0.035605  1.000000  0.000721  0.028975
  37036   0.085263  0.028293  0.000721  1.000000 -0.007267
  288146 -0.059640 -0.066133  0.028975 -0.007267  1.000000},
 1: {'cosine_similarity':         2310    962690  283390  129201  779699
  2310       1.0     1.0     1.0     1.0     1.0
  962690     1.0     1.0     1.0     1.0     1.0
  283390     1.0     1.0     1.0     1.0     1.0
  129201     1.0     1.0     1.0     1.0     1.0
  779699     1.0     1.0     1.0  

## Evaluation of recommendations

In [5]:
# Load the recipes data to map recipe names to their tags
recipes = pd.read_csv("data/cleaned_recipes_with_country.csv")
COLUMNS = ['time_tags', 'country_tags', 'dietary_tags', 'special_tags', 'ingredients_tags']

In [6]:
# Create a mapping from recipe name to its tags
recipe_tags_map = {}
for index, row in recipes.iterrows():
    tags = []
    for col in COLUMNS:
        tags.extend(row[col].strip('[]').replace("'", "").split(', '))
    tags = [tag.strip() for tag in tags if tag.strip()]
    recipe_tags_map[row['name']] = tags

In [7]:
# Evaluation Metrics
def happiness_score(recommendations, user_profiles):
    happiness_scores = []

    for group_id, recs in enumerate(recommendations):
        group_happiness = []

        for user in groups[group_id]:
            user_happiness = []
            for rec in recs:
                tags = recipe_tags_map.get(rec, [])
                individual_happiness = np.mean([user_profiles[user].get(tag, 0) for tag in tags])
                user_happiness.append(individual_happiness)

            group_happiness.append(np.mean(user_happiness))

        happiness_scores.append(np.mean(group_happiness))

    return happiness_scores

def fairness_score(recommendations, user_profiles):
    fairness_scores = []

    for group_id, recs in enumerate(recommendations):
        user_happiness = []

        for user in groups[group_id]:
            happiness = []
            for rec in recs:
                tags = recipe_tags_map.get(rec, [])
                individual_happiness = np.mean([user_profiles[user].get(tag, 0) for tag in tags])
                happiness.append(individual_happiness)

            user_happiness.append(np.mean(happiness))

        fairness_scores.append(np.std(user_happiness))

    return fairness_scores

def consensus_score(recommendations, user_profiles):
    consensus_scores = []

    for group_id, recs in enumerate(recommendations):
        group_consensus = []

        for rec in recs:
            tags = recipe_tags_map.get(rec, [])
            tag_ratings = np.array([[user_profiles[user].get(tag, 0) for tag in tags] for user in groups[group_id]])
            consensus = np.mean(np.std(tag_ratings, axis=0))
            group_consensus.append(consensus)

        consensus_scores.append(np.mean(group_consensus))

    return consensus_scores

def coverage_score(recommendations, user_profiles):
    coverage_scores = []

    for group_id, recs in enumerate(recommendations):
        all_tags = set()
        highly_rated_tags = set()

        for user in groups[group_id]:
            all_tags.update(user_profiles[user].keys())
            highly_rated_tags.update([tag for tag, rating in user_profiles[user].items() if rating >= 4])

        recommended_tags = set()
        for rec in recs:
            recommended_tags.update(recipe_tags_map.get(rec, []))

        coverage_scores.append(len(recommended_tags.intersection(highly_rated_tags)) / len(all_tags))

    return coverage_scores

def dcg_score(recommendations, user_profiles):
    dcg_scores = []

    for group_id, recs in enumerate(recommendations):
        group_dcg = []

        for user in groups[group_id]:
            user_dcg = 0
            for i, rec in enumerate(recs):
                tags = recipe_tags_map.get(rec, [])
                individual_happiness = np.mean([user_profiles[user].get(tag, 0) for tag in tags])
                user_dcg += (2**individual_happiness - 1) / np.log2(i + 2)
            group_dcg.append(user_dcg)

        dcg_scores.append(np.mean(group_dcg))

    return dcg_scores

def idcg_score(recommendations, user_profiles):
    idcg_scores = []

    for group_id, recs in enumerate(recommendations):
        group_idcg = []

        for user in groups[group_id]:
            user_idcg = 0
            sorted_recs = sorted(recs, key=lambda rec: np.mean([user_profiles[user].get(tag, 0) for tag in recipe_tags_map.get(rec, [])]), reverse=True)
            for i, rec in enumerate(sorted_recs):
                tags = recipe_tags_map.get(rec, [])
                individual_happiness = np.mean([user_profiles[user].get(tag, 0) for tag in tags])
                user_idcg += (2**individual_happiness - 1) / np.log2(i + 2)
            group_idcg.append(user_idcg)

        idcg_scores.append(np.mean(group_idcg))

    return idcg_scores

def ndcg_score(recommendations, user_profiles):
    dcg_scores = dcg_score(recommendations, user_profiles)
    idcg_scores = idcg_score(recommendations, user_profiles)
    ndcg_scores = [dcg / idcg if idcg != 0 else 0 for dcg, idcg in zip(dcg_scores, idcg_scores)]
    return ndcg_scores


In [8]:
# Get groups and compute metrics
groups = get_groups(groups_exp_1)

# Separate the recommendations for different aggregation methods
average_recommendations = recommendations_exp_1[0]
least_misery_recommendations = recommendations_exp_1[1]
most_pleasure_recommendations = recommendations_exp_1[2]

# Calculating the metrics for each method
happiness_scores_avg = happiness_score(average_recommendations, user_profiles)
fairness_scores_avg = fairness_score(average_recommendations, user_profiles)
consensus_scores_avg = consensus_score(average_recommendations, user_profiles)
coverage_scores_avg = coverage_score(average_recommendations, user_profiles)
ndcg_scores_avg = ndcg_score(average_recommendations, user_profiles)

happiness_scores_lm = happiness_score(least_misery_recommendations, user_profiles)
fairness_scores_lm = fairness_score(least_misery_recommendations, user_profiles)
consensus_scores_lm = consensus_score(least_misery_recommendations, user_profiles)
coverage_scores_lm = coverage_score(least_misery_recommendations, user_profiles)
ndcg_scores_lm = ndcg_score(least_misery_recommendations, user_profiles)

happiness_scores_mp = happiness_score(most_pleasure_recommendations, user_profiles)
fairness_scores_mp = fairness_score(most_pleasure_recommendations, user_profiles)
consensus_scores_mp = consensus_score(most_pleasure_recommendations, user_profiles)
coverage_scores_mp = coverage_score(most_pleasure_recommendations, user_profiles)
ndcg_scores_mp = ndcg_score(most_pleasure_recommendations, user_profiles)

# Display the results for each method
metrics_avg = pd.DataFrame({
    'Group': list(groups.keys()),
    'Happiness Score': happiness_scores_avg,
    'Fairness Score': fairness_scores_avg,
    'Consensus Score': consensus_scores_avg,
    'Coverage Score': coverage_scores_avg,
    'nDCG Score': ndcg_scores_avg
})

metrics_lm = pd.DataFrame({
    'Group': list(groups.keys()),
    'Happiness Score': happiness_scores_lm,
    'Fairness Score': fairness_scores_lm,
    'Consensus Score': consensus_scores_lm,
    'Coverage Score': coverage_scores_lm,
    'nDCG Score': ndcg_scores_lm
})

metrics_mp = pd.DataFrame({
    'Group': list(groups.keys()),
    'Happiness Score': happiness_scores_mp,
    'Fairness Score': fairness_scores_mp,
    'Consensus Score': consensus_scores_mp,
    'Coverage Score': coverage_scores_mp,
    'nDCG Score': ndcg_scores_mp
})

metrics_avg, metrics_lm, metrics_mp

(    Group  Happiness Score  Fairness Score  Consensus Score  Coverage Score  \
 0       0         4.090150        0.166098         0.618930        0.086096   
 1       1         4.151794        0.174198         0.616839        0.088879   
 2       2         3.539878        0.421654         0.870829        0.098214   
 3       3         3.807873        0.248005         0.723046        0.077308   
 4       4         3.941497        0.370582         0.836816        0.089584   
 5       5         3.883534        0.229595         0.667850        0.091284   
 6       6         3.950635        0.281256         0.677365        0.102526   
 7       7         3.673412        0.401567         0.816233        0.083884   
 8       8         3.905571        0.210426         0.597165        0.084887   
 9       9         4.171771        0.112775         0.511486        0.095341   
 10     10         3.785828        0.241332         0.716456        0.093010   
 11     11         3.870061        0.220

In [10]:
# Calculate and print the average for all groups
avg_metrics_avg = metrics_avg.mean()
avg_metrics_lm = metrics_lm.mean()
avg_metrics_mp = metrics_mp.mean()

print("Average Metrics for Average Aggregation Strategy")
print(avg_metrics_avg)

print("\nAverage Metrics for Least Misery Aggregation Strategy")
print(avg_metrics_lm)

print("\nAverage Metrics for Most Pleasure Aggregation Strategy")
print(avg_metrics_mp)

Average Metrics for Average Aggregation Strategy
Group              19.500000
Happiness Score     3.893344
Fairness Score      0.272428
Consensus Score     0.700027
Coverage Score      0.087830
nDCG Score          0.922237
dtype: float64

Average Metrics for Least Misery Aggregation Strategy
Group              19.500000
Happiness Score     3.893344
Fairness Score      0.272428
Consensus Score     0.700027
Coverage Score      0.087830
nDCG Score          0.922237
dtype: float64

Average Metrics for Most Pleasure Aggregation Strategy
Group              19.500000
Happiness Score     4.102899
Fairness Score      0.227505
Consensus Score     0.591934
Coverage Score      0.106848
nDCG Score          0.941501
dtype: float64


## Explore why average and least misery have the same values

In [11]:
# Sample debug prints to compare recommendations
print("Sample recommendations for Average Aggregation Strategy:")
for group_recs in average_recommendations[:2]:  # Printing first 2 groups for brevity
    print(group_recs)

print("\nSample recommendations for Least Misery Aggregation Strategy:")
for group_recs in least_misery_recommendations[:2]:  # Printing first 2 groups for brevity
    print(group_recs)

print("\nSample recommendations for Most Pleasure Aggregation Strategy:")
for group_recs in most_pleasure_recommendations[:2]:  # Printing first 2 groups for brevity
    print(group_recs)

Sample recommendations for Average Aggregation Strategy:
['copycat p f  chang s singapore street noodles', 'pappadeaux snapper ponchartrain', 'skips chili seasoning mix', 'gourmet bangers   mash', 'oven  fried  fish', 'pub style fish and chips', 'szechuan style eggplants', 'beef and scallops', 'beer battered fish with tartar sauce', 'fish fillet with garlic butter and black olives']
['delicious grilled chicken marinade', 'mesa burgers with sage aioli and spicy chips', 'cherry sauce for grilled salmon', 'grandma s grape jelly', 'kicked up fish cakes', 'pina banana colada', 'party punch ice ring', 'cape malay mango atjar   south african mango chutney', 'chicken ala mayo', 'maple roasted turkey  smoky sage cornbread stuffing   gravy']

Sample recommendations for Least Misery Aggregation Strategy:
['copycat p f  chang s singapore street noodles', 'pappadeaux snapper ponchartrain', 'skips chili seasoning mix', 'gourmet bangers   mash', 'oven  fried  fish', 'pub style fish and chips', 'szech

The average and least misery strategy recommend the same recipes